# 初始化

In [ ]:
import os
from local_packages import *                                                                                                                                                                                                                                                                                                                                               
from Packages.Multi_Process.multi_process import MultiProcessor

# 初始化API
agentopener=AgentOpener(service_type='qwen', version='long')

llm=agentopener.service

# 初始化向量嵌入
embedder=QwenEmbedder()

# 初始化文字分割器
divider=TextDivider(threshold=4096,overlap=512)
processor=LLMParser()

def transform_tuple_list(data):
    transformed_data = []
    for string_list, index in data:
        for string in string_list:
            transformed_data.append((string, index))
    return transformed_data

def string2tuple_list(string_list):
    """
    将字符串列表转换为包含元组的列表，每个元组包含一个字符串及其索引。

    参数:
    string_list (list of str): 输入的字符串列表。

    返回:
    list of tuple: 包含元组的列表，每个元组是 (string, index)。
    """
    tuple_list = [(string, index) for index, string in enumerate(string_list)]
    return tuple_list

def tuple2string_list(tuple_list):
    """
    将包含元组的列表转换为字符串列表，每个元组包含一个字符串及其索引。

    参数:
    tuple_list (list of tuple): 输入的包含元组的列表，每个元组是 (string, index)。

    返回:
    list of str: 字符串列表。
    """
    string_list = [item[0] for item in tuple_list]
    return string_list

def update_tuple_list(tuple_list, synonyms):
    # 创建一个映射，将每个类似词指向它的main_word
    synonym_map = {}
    for main_word, similar_dict in synonyms.items():
        synonym_map[main_word] = main_word  # 主词也需要指向自己
        for similar_word in similar_dict['Similar_keys']:
            synonym_map[similar_word] = main_word

    # 更新tuple_list中的字符串，并避免重复
    updated_tuple_list = []
    seen = set()  # 用于跟踪已经添加的元素
    for string, index in tuple_list:
        main_word = synonym_map.get(string, string)
        element = (main_word, index)
        if element not in seen:
            updated_tuple_list.append(element)
            seen.add(element)
    
    return updated_tuple_list


# 类别析取模板

In [70]:
data_template02_entity_type_list=f'''
['实体类型1',...,'实体类型n']
'''

prompt02_entity_type_list='''
你是一个聪明的助手，你的工作是帮助人们理解文本中的信息。
当给定一段文字时，你的任务是为这段文字中的实体进行分类。
非常重要：不要生成重复或重叠的实体类型。例如，如果文本包含"公司"和"组织"，你应该只返回其中一个。
数量不是重点，质量优先。确保你的答案中的每一个内容都与实体提取的上下文相关。
记住，我们需要的是实体类型。
接下来是正式的任务：
--------------------------------------------------------------------------------
请根据给定的文本，给出其中涉及的实体类型。
对于这一段文字:

{input_1}

我希望你识别出它所涉及到的主题，并且采用下面的格式回复：

{data_template}
'''

def validation02_entity_type_list(data):
    """
    检查数据是否符合[string, ..., string]的格式。

    参数:
        data (list): 待检查的数据。

    返回:
        bool: 如果数据符合格式则返回True，否则返回False。
    """
    if not isinstance(data, list):
        return False
    
    for item in data:
        if not isinstance(item, str):
            return False
    
    return True

correction02_entity_type_list='''
下列内容中含有一个错误的数据格式：

{answer}

请你修改它，使其符合以下格式：

{data_template}
'''


# 实体查找模板

In [69]:
data_template03_entity_extract=f'''
['实体1',...,'实体n']
'''

prompt03_entity_extract='''
你是一个聪明的助手，你的工作是帮助人们理解文本中的信息。
给定一个可能与这个活动相关的文本，请你完整、详细、毫无遗漏地找出文本中的所有实体。
请你找出所有实体。对每个找到的实体，提取以下信息：
   - 实体名称：原文中的实体名称

--------------------------------------------------------------------------------
请根据给定的文本，给出其中提到的实体。
对于这一段文字:

{input_1}

我希望你完整、详细、毫无遗漏地抽取出它其中的所有实体，并且采用下面的格式回复：

{data_template}
'''

def validation03_entity_extract(data):
    """
    检查数据是否符合
    {string: {'相关性分数': float, '实体类型': string}, ..., string: {'相关性分数': float, '实体类型': string}}
    的格式。

    参数:
        data (dict): 待检查的数据。

    返回:
        bool: 如果数据符合格式则返回True，否则返回False。
    """
    if not isinstance(data, list):
        return False
    
    for item in data:
        if not isinstance(item, str):
            return False
    
    return True

correction03_entity_extract='''
下列内容中含有一个错误的数据格式：

{answer}

请你修改它，使其符合以下格式：

{data_template}
'''

# 主工作流的第一部分，得到近义词合并后的实体和类型

In [71]:
divider.threshold=128
divider.overlap=16

input_chunk_list=divider.divide(txt_path='test.txt')

# 列表中的元素大部分时候最好是带着索引，因为在多线程阶段很可能需要根据索引来处理
# 但是大部分的处理函数并没有很良好地兼容索引(过于特化)，在对付不含多线程的任务时，为了降低复杂度，转换回最一般的字符串列表还是有必要的
input_chunk_tuple_list=string2tuple_list(input_chunk_list)
# 输入元组
entity_classifier=MultiProcessor(llm,processor, data_template02_entity_type_list, prompt02_entity_type_list, correction02_entity_type_list, validation02_entity_type_list, isList=True)

type_tuple_list=entity_classifier.multitask_perform(input_chunk_tuple_list, num_threads=20)
# 输入元组
entity_extractor=MultiProcessor(llm, processor, data_template03_entity_extract, prompt03_entity_extract, correction03_entity_extract, validation03_entity_extract, isList=True)

entity_tuple_list=entity_extractor.multitask_perform(input_chunk_tuple_list, num_threads=20)



{'input_1': '百度APP中的【百度百科】小程序\n体验更全面的产品功能，享受更高质量的浏览体验'} ['软件应用', '网络服务', '科技公司']
result (['软件应用', '网络服务', '科技公司'], 2)
{'input_1': '人\n几乎涵盖了所有已知的知识领域。\n“世界很复杂，百度更懂你”，百度百科旨在创造一个涵盖各领域知识的中文信息收集平台。百度 …\n\n2024年4月，百度百科团队决定于2024年6月30日起\n将百度百科APP的全部功能升级、迁移至百度APP内\n使用百度APP中的【百度百科】小程序'} ['组织', '日期', '软件应用']
result (['组织', '日期', '软件应用'], 1)
{'input_1': '百度公司推出的网络百科全书\n\n百度百科是百度公司推出的一部内容开放\n自由的网络百科全书。其测试版于2006年4月20日上线\n正式版在2008年4月21日发布\n截至2023年4月，百度百科已经收录了超2700万个词条\n参与词条编辑的网友超过770万人\n几乎涵盖了所有已知的知识领域。'} ['公司', '产品', '日期', '数量']
result (['公司', '产品', '日期', '数量'], 0)
{'input_1': '百度APP中的【百度百科】小程序\n体验更全面的产品功能，享受更高质量的浏览体验'} ['百度APP', '百度百科']
result (['百度APP', '百度百科'], 2)
{'input_1': '人\n几乎涵盖了所有已知的知识领域。\n“世界很复杂，百度更懂你”，百度百科旨在创造一个涵盖各领域知识的中文信息收集平台。百度 …\n\n2024年4月，百度百科团队决定于2024年6月30日起\n将百度百科APP的全部功能升级、迁移至百度APP内\n使用百度APP中的【百度百科】小程序'} ['百度', '百度百科', '百度APP', '百度百科团队', '2024年4月', '2024年6月30日', '小程序']
result (['百度', '百度百科', '百度APP', '百度百科团队', '2024年4月', '2024年6月30日', '小程序'], 1)
{'input_1': '百度公司推出的网络百科全书\n\n百度百科是

In [72]:
def convertor(tuple_list):
    temp_type_list=transform_tuple_list(tuple_list)

    type_list=tuple2string_list(temp_type_list)

    embedded_type_list=embedder.embed_list(type_list)

    synonyms=embedder.partition_by_similarity(embedded_type_list)

    updated_type_list=update_tuple_list(temp_type_list, synonyms)
    
    return updated_type_list

In [73]:
merged_type_tuple_list=convertor(type_tuple_list)
merged_entity_tuple_list=convertor(entity_tuple_list)

# 整理数据结构，以待接下来的挂载

In [74]:
def organize_data(type_data, entity_data):
    # 初始化一个字典，用于存储按 index 分组的数据
    organized_data = {}

    # 处理 type_data
    for type_string, index in type_data:
        if index not in organized_data:
            organized_data[index] = {'type_list': [], 'entity_list': []}
        organized_data[index]['type_list'].append(type_string)

    # 处理 entity_data
    for entity_string, index in entity_data:
        if index not in organized_data:
            organized_data[index] = {'type_list': [], 'entity_list': []}
        organized_data[index]['entity_list'].append(entity_string)

    # 将字典转换为列表形式，包含 (type_list, entity_list, index) 的元组
    result = [(data['type_list'], data['entity_list'], index) for index, data in organized_data.items()]

    return result
# 组织后的数据
organized_data = organize_data(merged_type_tuple_list,merged_entity_tuple_list)

print(organized_data)

[(['公司', '产品', '日期', '数量'], ['百度公司', '百度百科', '网络百科全书', '2006年4月20日', '2008年4月21日', '2023年4月', '2700万个词条', '770万人', '知识领域'], 0), (['组织', '日期', '软件应用'], ['百度百科', '百度APP', '百度百科团队', '2023年4月', '2024年6月30日', '小程序'], 1), (['软件应用', '网络服务', '科技公司'], ['百度APP', '百度百科'], 2)]


# tag的挂载，这是平行的

In [67]:
data_template04_mount_tag='''
[
{
    "名称": name_str,
    "类型": type_str,
    "相关度": float
},...,
]
'''

prompt04_mount_tag='''
现在有一些实体{input_2}
我们关心其中属于{input_1}的实体，
请你帮我进行这个归类，并为其打上相关度分数，这个分数是一个浮点数，位于0到10之间，以如下结构返回:
{data_template}
'''

def validation04_mount_tag(data):
    if not isinstance(data, list):
        return False
    
    for item in data:
        if not isinstance(item, dict):
            return False
        if '名称' not in item or not isinstance(item['名称'], str):
            return False
        if '类型' not in item or not isinstance(item['类型'], str):
            return False
        if '相关度' not in item or not isinstance(item['相关度'], float):
            return False
    
    return True

correction04_mount_tag='''
下列内容中含有一个错误的数据格式：

{answer}

请你修改它，使其符合以下格式：

{data_template}
'''

In [75]:
tag_mounter=MultiProcessor(llm, processor, data_template04_mount_tag, prompt04_mount_tag, correction04_mount_tag, validation04_mount_tag, isList=True)
result=tag_mounter.multitask_perform(organized_data, num_threads=40)
print(result)

{'input_1': ['软件应用', '网络服务', '科技公司'], 'input_2': ['百度APP', '百度百科']} [{'名称': '百度APP', '类型': '软件应用', '相关度': 9.5}, {'名称': '百度APP', '类型': '网络服务', '相关度': 8.5}, {'名称': '百度百科', '类型': '网络服务', '相关度': 9.0}, {'名称': '百度百科', '类型': '科技公司', '相关度': 7.0}]
result ([{'名称': '百度APP', '类型': '软件应用', '相关度': 9.5}, {'名称': '百度APP', '类型': '网络服务', '相关度': 8.5}, {'名称': '百度百科', '类型': '网络服务', '相关度': 9.0}, {'名称': '百度百科', '类型': '科技公司', '相关度': 7.0}], 2)
{'input_1': ['组织', '日期', '软件应用'], 'input_2': ['百度百科', '百度APP', '百度百科团队', '2023年4月', '2024年6月30日', '小程序']} [{'名称': '百度百科', '类型': '组织', '相关度': 9.5}, {'名称': '百度APP', '类型': '软件应用', '相关度': 10.0}, {'名称': '百度百科团队', '类型': '组织', '相关度': 9.0}, {'名称': '2023年4月', '类型': '日期', '相关度': 8.0}, {'名称': '2024年6月30日', '类型': '日期', '相关度': 8.5}, {'名称': '小程序', '类型': '软件应用', '相关度': 7.5}]
result ([{'名称': '百度百科', '类型': '组织', '相关度': 9.5}, {'名称': '百度APP', '类型': '软件应用', '相关度': 10.0}, {'名称': '百度百科团队', '类型': '组织', '相关度': 9.0}, {'名称': '2023年4月', '类型': '日期', '相关度': 8.0}, {'名称': '2024年6月30日', '类型': '日期', '相关度

# 同时获取着实体的定义

In [77]:
print(input_chunk_tuple_list)
print(merged_entity_tuple_list)
explanation_data=organize_data(input_chunk_tuple_list, merged_entity_tuple_list)

[('百度公司推出的网络百科全书\n\n百度百科是百度公司推出的一部内容开放\n自由的网络百科全书。其测试版于2006年4月20日上线\n正式版在2008年4月21日发布\n截至2023年4月，百度百科已经收录了超2700万个词条\n参与词条编辑的网友超过770万人\n几乎涵盖了所有已知的知识领域。', 0), ('人\n几乎涵盖了所有已知的知识领域。\n“世界很复杂，百度更懂你”，百度百科旨在创造一个涵盖各领域知识的中文信息收集平台。百度 …\n\n2024年4月，百度百科团队决定于2024年6月30日起\n将百度百科APP的全部功能升级、迁移至百度APP内\n使用百度APP中的【百度百科】小程序', 1), ('百度APP中的【百度百科】小程序\n体验更全面的产品功能，享受更高质量的浏览体验', 2)]
[('百度公司', 0), ('百度百科', 0), ('网络百科全书', 0), ('2006年4月20日', 0), ('2008年4月21日', 0), ('2023年4月', 0), ('2700万个词条', 0), ('770万人', 0), ('知识领域', 0), ('百度百科', 1), ('百度APP', 1), ('百度百科团队', 1), ('2023年4月', 1), ('2024年6月30日', 1), ('小程序', 1), ('百度APP', 2), ('百度百科', 2)]


In [80]:
data_template05_entity_explain='''
[
{
    "名称": name_str,
    "解释": type_str
},...,
]
'''

prompt05_entity_explain='''
现在我有一些实体{input_2}
他们出自这个文段{input_1}，
请你帮我依据本文段给出这些实体的解释，以如下结构返回:
{data_template}
'''

def validation05_entity_explain(data):
    if not isinstance(data, list):
        return False
    
    for item in data:
        if not isinstance(item, dict):
            return False
        if '名称' not in item or not isinstance(item['名称'], str):
            return False
        if '解释' not in item or not isinstance(item['解释'], str):
            return False
    
    return True

correction05_entity_explain='''
下列内容中含有一个错误的数据格式：

{answer}

请你修改它，使其符合以下格式：

{data_template}
'''
entity_explainer=MultiProcessor(llm, processor, data_template05_entity_explain, prompt05_entity_explain, correction05_entity_explain, validation05_entity_explain, isList=True)

In [81]:
entity_explanation=entity_explainer.multitask_perform(explanation_data, num_threads=40)
print(entity_explanation)

{'input_1': ['百度APP中的【百度百科】小程序\n体验更全面的产品功能，享受更高质量的浏览体验'], 'input_2': ['百度APP', '百度百科']} [{'名称': '百度APP', '解释': '一款集搜索、资讯、视频、社交等多种功能于一体的移动应用程序,由百度公司开发.'}, {'名称': '百度百科', '解释': '百度旗下的一款在线百科全书服务,提供海量的知识条目,用户可以在其中查找和学习各种信息.在百度APP中,通过[百度百科]小程序可以体验更全面的产品功能,享受更高质量的浏览体验.'}]
result ([{'名称': '百度APP', '解释': '一款集搜索、资讯、视频、社交等多种功能于一体的移动应用程序,由百度公司开发.'}, {'名称': '百度百科', '解释': '百度旗下的一款在线百科全书服务,提供海量的知识条目,用户可以在其中查找和学习各种信息.在百度APP中,通过[百度百科]小程序可以体验更全面的产品功能,享受更高质量的浏览体验.'}], 2)
{'input_1': ['人\n几乎涵盖了所有已知的知识领域。\n“世界很复杂，百度更懂你”，百度百科旨在创造一个涵盖各领域知识的中文信息收集平台。百度 …\n\n2024年4月，百度百科团队决定于2024年6月30日起\n将百度百科APP的全部功能升级、迁移至百度APP内\n使用百度APP中的【百度百科】小程序'], 'input_2': ['百度百科', '百度APP', '百度百科团队', '2023年4月', '2024年6月30日', '小程序']} [{'名称': '百度百科', '解释': '一个涵盖各领域知识的中文信息收集平台,旨在提供全面的知识内容,几乎涵盖了所有已知的知识领域.'}, {'名称': '百度APP', '解释': '百度公司开发的一款应用软件,自2024年6月30日起,将整合百度百科的全部功能,用户可以在其中使用百度百科的小程序.'}, {'名称': '百度百科团队', '解释': '负责百度百科运营和管理的专业团队,他们在2024年4月决定将百度百科APP的功能升级并迁移至百度APP中.'}, {'名称': '2023年4月', '解释': '时间点,但文中未提及具体与该时间点相关的信息或事件.'}, {'名称': '2024年6

# 获取关系

In [83]:
data_template06_relations='''
[
{
    "实体名称1": name_str,
    "实体名称2": name_str,
    "关系类型": type_str,
    "关系解释": expr_str,
    "相关程度分数": float
},...,
]
'''

prompt06_relations='''

你是一个聪明的助手，你的工作是帮助人们理解文本中的信息。
给定一个可能与这个活动相关的文本{input_1}和一个实体列表{input_2}，请你指出实体列表中有相关性的实体
对于找到的每一对相关的实体，提取以下信息：
   - 实体名称1：相关实体对中更主要的那个实体
   - 实体名称2：相关实体对中相对次要的那个实体
   - 关系类型：他们之间的关系类型
   - 关系解释：这个关系类型的解释
   - 相关程度分数：关于这个实体对两个实体之间的相关性打分，分数是0到10之间的浮点数，1表示微不足道或不相关，10表示具有高度和深刻的相关性
接下来是正式的任务：
--------------------------------------------------------------------------------
现在我有一些实体{input_2}
他们出自这个文段{input_1}，
请你帮我依据本文段给出这些实体的相关对，以如下数据结构返回:
{data_template}
'''

def validation06_relations(data):
    if not isinstance(data, list):
        return False
    
    for item in data:
        if not isinstance(item, dict):
            return False
        if '实体名称1' not in item or not isinstance(item['实体名称1'], str):
            return False
        if '实体名称2' not in item or not isinstance(item['实体名称2'], str):
            return False
        if '关系类型' not in item or not isinstance(item['关系类型'], str):
            return False
        if '关系解释' not in item or not isinstance(item['关系解释'], str):
            return False
        if '相关程度分数' not in item or not isinstance(item['相关程度分数'], (int, float)):
            return False
    
    return True

correction06_relations='''
下列内容中含有一个错误的数据格式：

{answer}

请你修改它，使其符合以下格式：

{data_template}
'''

relation_explainer=MultiProcessor(llm, processor, data_template06_relations, prompt06_relations, correction06_relations, validation06_relations, isList=True)

In [92]:
relation_explanation=relation_explainer.multitask_perform(explanation_data, num_threads=40)
print(relation_explanation)

{'input_1': ['百度APP中的【百度百科】小程序\n体验更全面的产品功能，享受更高质量的浏览体验'], 'input_2': ['百度APP', '百度百科']} [{'实体名称1': '百度APP', '实体名称2': '百度百科', '关系类型': '包含关系', '关系解释': '百度APP中包含[百度百科]小程序,用户可以通过百度APP体验百度百科的内容和服务.', '相关程度分数': 9.5}]
result ([{'实体名称1': '百度APP', '实体名称2': '百度百科', '关系类型': '包含关系', '关系解释': '百度APP中包含[百度百科]小程序,用户可以通过百度APP体验百度百科的内容和服务.', '相关程度分数': 9.5}], 2)
{'input_1': ['人\n几乎涵盖了所有已知的知识领域。\n“世界很复杂，百度更懂你”，百度百科旨在创造一个涵盖各领域知识的中文信息收集平台。百度 …\n\n2024年4月，百度百科团队决定于2024年6月30日起\n将百度百科APP的全部功能升级、迁移至百度APP内\n使用百度APP中的【百度百科】小程序'], 'input_2': ['百度百科', '百度APP', '百度百科团队', '2023年4月', '2024年6月30日', '小程序']} [{'实体名称1': '百度百科', '实体名称2': '百度APP', '关系类型': '功能迁移', '关系解释': '百度百科的功能将被迁移到百度APP内,作为其一部分提供服务.', '相关程度分数': 9.5}, {'实体名称1': '百度百科团队', '实体名称2': '2024年6月30日', '关系类型': '时间决策', '关系解释': '百度百科团队决定在2024年6月30日起执行功能迁移计划.', '相关程度分数': 9.0}, {'实体名称1': '百度APP', '实体名称2': '小程序', '关系类型': '包含关系', '关系解释': '百度APP内包含了名为[百度百科]的小程序,用于提供百度百科的服务.', '相关程度分数': 8.5}]
result ([{'实体名称1': '百度百科', '实体名称2': '百度APP', '关系类型': '功能迁移', '关系解释': '百度百科的功能将被迁

实际上到这里图搭建就已经完成了，接下来只需要对 原始文段块、实体类型、实体解释、相关关系 做一轮融合就结束了

原始文段块：

In [93]:
input_chunk_tuple_list

[('百度公司推出的网络百科全书\n\n百度百科是百度公司推出的一部内容开放\n自由的网络百科全书。其测试版于2006年4月20日上线\n正式版在2008年4月21日发布\n截至2023年4月，百度百科已经收录了超2700万个词条\n参与词条编辑的网友超过770万人\n几乎涵盖了所有已知的知识领域。',
  0),
 ('人\n几乎涵盖了所有已知的知识领域。\n“世界很复杂，百度更懂你”，百度百科旨在创造一个涵盖各领域知识的中文信息收集平台。百度 …\n\n2024年4月，百度百科团队决定于2024年6月30日起\n将百度百科APP的全部功能升级、迁移至百度APP内\n使用百度APP中的【百度百科】小程序',
  1),
 ('百度APP中的【百度百科】小程序\n体验更全面的产品功能，享受更高质量的浏览体验', 2)]

实体类型

In [101]:
merged_entity_tuple_list

[('百度公司', 0),
 ('百度百科', 0),
 ('网络百科全书', 0),
 ('2006年4月20日', 0),
 ('2008年4月21日', 0),
 ('2023年4月', 0),
 ('2700万个词条', 0),
 ('770万人', 0),
 ('知识领域', 0),
 ('百度百科', 1),
 ('百度APP', 1),
 ('百度百科团队', 1),
 ('2023年4月', 1),
 ('2024年6月30日', 1),
 ('小程序', 1),
 ('百度APP', 2),
 ('百度百科', 2)]

实体解释

In [95]:
entity_explanation

[([{'名称': '百度公司',
    '解释': '中国的一家互联网巨头,专注于搜索引擎、云计算和人工智能技术,推出了包括百度百科在内的多种互联网产品和服务.'},
   {'名称': '百度百科', '解释': '百度公司推出的一部内容开放、自由的网络百科全书,用户可以自由编辑和查阅各种知识条目.'},
   {'名称': '网络百科全书', '解释': '一种在线的参考工具,包含大量由用户贡献或专业编辑维护的信息条目,覆盖广泛的主题和知识领域.'},
   {'名称': '2006年4月20日', '解释': '百度百科测试版上线的时间,标志着该网络百科全书项目的初步启动.'},
   {'名称': '2008年4月21日', '解释': '百度百科正式版发布的时间,项目从测试阶段过渡到全面运营阶段.'},
   {'名称': '2023年4月', '解释': '百度百科发展的一个时间节点,用于描述其收录词条数量和参与编辑用户规模的状态.'},
   {'名称': '2700万个词条', '解释': '截至2023年4月,百度百科收录的词条总数,反映了其内容的丰富性和覆盖面之广.'},
   {'名称': '770万人', '解释': '截至2023年4月,参与百度百科词条编辑的网友人数,体现了其社区的活跃度和用户参与度.'},
   {'名称': '知识领域', '解释': '百度百科覆盖的各种学科和主题范围,几乎包括所有已知的知识领域,如科学、艺术、历史等.'}],
  0),
 ([{'名称': '百度百科', '解释': '一个涵盖各领域知识的中文信息收集平台,旨在提供全面的知识内容,几乎涵盖了所有已知的知识领域.'},
   {'名称': '百度APP',
    '解释': '百度公司开发的一款应用软件,自2024年6月30日起,将整合百度百科的全部功能,用户可以在其中使用百度百科的小程序.'},
   {'名称': '百度百科团队',
    '解释': '负责百度百科运营和管理的专业团队,他们在2024年4月决定将百度百科APP的功能升级并迁移至百度APP中.'},
   {'名称': '2023年4月', '解释': '时间点,但文中未提及具体与该时间点相关的信息或事件.'},
   {'名称': '2024年6月30日', '解释': 

关系解释

In [96]:
relation_explanation

[([{'实体名称1': '百度公司',
    '实体名称2': '百度百科',
    '关系类型': '创建者与产品',
    '关系解释': '百度公司创建了百度百科这一网络百科全书.',
    '相关程度分数': 10.0},
   {'实体名称1': '百度百科',
    '实体名称2': '网络百科全书',
    '关系类型': '实例与类别',
    '关系解释': '百度百科是网络百科全书的一个具体实例.',
    '相关程度分数': 10.0},
   {'实体名称1': '百度百科',
    '实体名称2': '2006年4月20日',
    '关系类型': '事件时间',
    '关系解释': '2006年4月20日是百度百科测试版上线的时间.',
    '相关程度分数': 9.5},
   {'实体名称1': '百度百科',
    '实体名称2': '2008年4月21日',
    '关系类型': '事件时间',
    '关系解释': '2008年4月21日是百度百科正式版发布的日期.',
    '相关程度分数': 9.5},
   {'实体名称1': '百度百科',
    '实体名称2': '2023年4月',
    '关系类型': '事件时间',
    '关系解释': '2023年4月是百度百科达到特定里程碑(词条数量)的时间点.',
    '相关程度分数': 9.0},
   {'实体名称1': '百度百科',
    '实体名称2': '2700万个词条',
    '关系类型': '包含',
    '关系解释': '百度百科截至2023年4月收录了超过2700万个词条.',
    '相关程度分数': 10.0},
   {'实体名称1': '百度百科',
    '实体名称2': '770万人',
    '关系类型': '参与者数量',
    '关系解释': '有超过770万人参与了百度百科词条的编辑.',
    '相关程度分数': 10.0},
   {'实体名称1': '百度百科',
    '实体名称2': '知识领域',
    '关系类型': '覆盖范围',
    '关系解释': '百度百科几乎涵盖了所有已知的知识领域.',
    '相关程度分数': 10.0}],
  

In [99]:
def convert_structure(data):
    result = {}

    for item_list, index in data:
        for item in item_list:
            name = item['名称']
            explanation = item['解释']
            if name not in result:
                result[name] = []
            result[name].append((explanation, index))

    return result

# 转换数据结构
converted_data = convert_structure(entity_explanation)


In [100]:
converted_data

{'百度公司': [('中国的一家互联网巨头,专注于搜索引擎、云计算和人工智能技术,推出了包括百度百科在内的多种互联网产品和服务.', 0)],
 '百度百科': [('百度公司推出的一部内容开放、自由的网络百科全书,用户可以自由编辑和查阅各种知识条目.', 0),
  ('一个涵盖各领域知识的中文信息收集平台,旨在提供全面的知识内容,几乎涵盖了所有已知的知识领域.', 1),
  ('百度旗下的一款在线百科全书服务,提供海量的知识条目,用户可以在其中查找和学习各种信息.在百度APP中,通过[百度百科]小程序可以体验更全面的产品功能,享受更高质量的浏览体验.',
   2)],
 '网络百科全书': [('一种在线的参考工具,包含大量由用户贡献或专业编辑维护的信息条目,覆盖广泛的主题和知识领域.', 0)],
 '2006年4月20日': [('百度百科测试版上线的时间,标志着该网络百科全书项目的初步启动.', 0)],
 '2008年4月21日': [('百度百科正式版发布的时间,项目从测试阶段过渡到全面运营阶段.', 0)],
 '2023年4月': [('百度百科发展的一个时间节点,用于描述其收录词条数量和参与编辑用户规模的状态.', 0),
  ('时间点,但文中未提及具体与该时间点相关的信息或事件.', 1)],
 '2700万个词条': [('截至2023年4月,百度百科收录的词条总数,反映了其内容的丰富性和覆盖面之广.', 0)],
 '770万人': [('截至2023年4月,参与百度百科词条编辑的网友人数,体现了其社区的活跃度和用户参与度.', 0)],
 '知识领域': [('百度百科覆盖的各种学科和主题范围,几乎包括所有已知的知识领域,如科学、艺术、历史等.', 0)],
 '百度APP': [('百度公司开发的一款应用软件,自2024年6月30日起,将整合百度百科的全部功能,用户可以在其中使用百度百科的小程序.', 1),
  ('一款集搜索、资讯、视频、社交等多种功能于一体的移动应用程序,由百度公司开发.', 2)],
 '百度百科团队': [('负责百度百科运营和管理的专业团队,他们在2024年4月决定将百度百科APP的功能升级并迁移至百度APP中.', 1)],
 '2024年6月30日': [('百度百科团队计划将百度百科APP的全部